In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from os import environ
from pathlib import Path
from typing import List

import torch
import pandas as pd
import numpy as np

from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from bliss.surveys.des import DarkEnergySurvey, DESDownloader

from pathlib import Path
from hydra import initialize, compose
from bliss.main import predict
import case_studies.galaxy_clustering.utils.diagnostics as diagnostics

In [ ]:
environ["CUDA_VISIBLE_DEVICES"] = "6"

In [ ]:
with initialize(config_path="../", version_base=None):
    cfg = compose("config", {
        "encoder.tiles_to_crop=0",
        })

In [ ]:
cfg.predict

In [ ]:
encoder = instantiate(cfg.predict.encoder)
enc_state_dict = torch.load(cfg.predict.weight_save_path)
enc_state_dict = enc_state_dict["state_dict"]
encoder.load_state_dict(enc_state_dict)
dataset = instantiate(cfg.cached_simulator)
trainer = instantiate(cfg.predict.trainer)

In [ ]:
cfg.predict.encoder

In [ ]:
enc_output = trainer.predict(encoder, datamodule=dataset)

In [ ]:
dataloader_iter = iter(dataset.predict_dataloader())

In [ ]:
enc_output[0]["mode_cat"]["membership"].shape

In [ ]:
truth = next(dataloader_iter)
outputs = enc_output[0]
accuracy = (outputs["mode_cat"]["membership"].to(torch.bool) == truth["tile_catalog"]["membership"].to(torch.bool)).sum()
print(accuracy/(2*10**2))

In [ ]:
(outputs["mode_cat"]["membership"] == truth["tile_catalog"]["membership"]).sum()

In [ ]:
truth["tile_catalog"]["membership"].shape

In [ ]:
batch_id = 1
dataloader_iter = iter(dataset.predict_dataloader())
for i in range(batch_id):
    truth = next(dataloader_iter)
    outputs = enc_output[i]

In [ ]:

# img_id selects for an image within the batch, i.e., from 0 to batch_size - 1
img_id = 0
crop = 0
n_tiles = 20
tile_size = 128
img_size = 1280

band_idx = {
    "g": 0,
    "r": 1,
    "i": 2,
    "z": 3
}


est_cat = outputs["mode_cat"]["membership"][img_id, crop: n_tiles - crop + 1, crop: n_tiles - crop + 1].to(torch.bool)
true_cat = truth["tile_catalog"]["membership"][img_id, crop: n_tiles - crop + 1, crop: n_tiles - crop + 1].to(torch.bool)

img_acc, img_prec, img_rec, img_f1 =  diagnostics.compute_metrics(est_cat, true_cat)

test_img = truth["images"][img_id][[band_idx["g"], band_idx["r"], band_idx["z"]]]
test_out = outputs["mode_cat"]["membership"][img_id].squeeze()
overlay = diagnostics.create_overlay(test_out, tile_size=tile_size)
blended_image = diagnostics.blend_images(test_img.permute(1,2,0), overlay, alpha=0.35, img_crop=crop*tile_size)

print(f"Image statistics:\n Accuracy:{img_acc}, \n Precision: {img_prec},\n Recall: {img_rec}, \n F1: {img_f1}")
plt.imshow(blended_image)
plt.axis('off')
plt.show()

#est_cat = est_cat.squeeze()
#true_cat = true_cat.squeeze()
#pred_heatmap = 2*est_cat + true_cat

#cmap = ListedColormap(['green', 'yellow', 'orange', 'lime'])

#plt.imshow(pred_heatmap, cmap=cmap)
#cbar = plt.colorbar()

#yticks = np.linspace(*cbar.ax.get_ylim(), cmap.N+1)[:-1]
#yticks += (yticks[1] - yticks[0]) / 2

# add tick labels to colorbar
#cbar.set_ticks(yticks, labels=['true negative','false negative','false positive','true positive'])
#cbar.ax.tick_params(length=0)
#plt.show()

In [ ]:
prob_array = enc_output[1]["parameters"].sigmoid().clamp(1e-4, 1 - 1e-4)[0].squeeze().numpy()
variance = prob_array * (1 - prob_array)

plt.figure(figsize=(6,6))
plt.imshow(prob_array, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Probability')
plt.title('Probability Heatmap')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

plt.figure(figsize=(6,6))
plt.imshow(variance, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Variance')
plt.title('Variance Heatmap')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

plt.tight_layout()